# Many Time Pad

- Let us see what goes wrong when a stream cipher key is used more than once. Below are eleven hex-encoded ciphertexts that are the result of encrypting eleven plaintexts with a stream cipher, all with the same stream cipher key. Your goal is to decrypt
the last ciphertext, and submit the secret message within it as solution.
- Hint: XOR the ciphertexts together, and consider what happens when a space is XORed with a character in a-z, A-Z

```Python
import sys
MSGS = ( --- 11 secret messages --- )

def strxor(a, b): # xor two strings of different lengths
    if len(a) > len(b):
        return "".join([chr(ord(x) ^ ord(y)) for (x, y) in zip(a[:len(b)], b)])
    else:
        return "".join([chr(ord(x) ^ ord(y)) for (x, y) in zip(a, b[:len(a)])])
        
def random(size=16):
    return open("/dev/urandom").read(size)

def encrypt(key, msg):
    c = strxor(key, msg)
    print (c.encode('hex'))
    return c

def main():
    key = random(1024)
    ciphertexts = [encrypt(key, msg) for msg in MSGS]
```

In [1]:
AZ = [chr(i) for i in range(ord('A'), ord('Z') + 1)]
az = [chr(i) for i in range(ord('a'), ord('z') + 1)]
AZhex = [hex(i)[2:4] for i in range(ord('A'), ord('Z') + 1)]
azhex = [hex(i)[2:4] for i in range(ord('A'), ord('Z') + 1)]
AZaz = AZ + az

lookupAZhex = {} 
lookupazhex = {}

for x, y in zip(AZhex, AZ):
    lookupAZhex[x] =y 

for x, y in zip(azhex, az):
    lookupazhex[x] =y

In [2]:
def xor_strhex(hex_string1, hex_string2): 
    # xor two strings of different lengths
    # two string are in HEX, truncate the longer string
    # the length of the shorter
    
    shorter_length = min(len(hex_string1), len(hex_string2))
    a, b = hex_string1[:shorter_length], hex_string2[:shorter_length]
    result = ""
    for x, y in zip(a, b):
        i, j = int(x, 16), int(y, 16)
        assert i in range(0, 16), "your string is not hex"
        assert j in range(0, 16), "your string is not hex"
        k = i ^ j
        result += hex(k)[2]

    return result
    
    
def process2analyze(message):
    result=''
    #Get two characters at a time
    for i in range(len(message) // 2):
        snippet = message[2*i:( 2*i + 2)]
        if snippet in lookupAZhex:
            result += lookupAZhex[snippet]
        elif snippet in lookupazhex:
            result += lookupazhex[snippet]
        else:
            result += " "
    
    return result

In [ ]:
# arguments are strings of same length
# apparently this is unnecessary haha 
def partial_decode(m1m2, m1m3, m2m3):
    m1, m2, m3 = '','',''
    
    for i in range(len(m1m2)):
        if m1m2[i] not in AZaz:
            if m1m3[i] in AZaz and m2m3[i] in AZaz:
                m1 += m1m3[i] # chr(ord(m1m3[i]) ^ ord(' ')) 
                m2 += m2m3[i] 
                m3 += "_"
            else: 
                m1+= " "
                m2+= " "
                m3+= " "
        elif m1m3[i] not in AZaz:
            if m1m2[i] in AZaz and m2m3[i] in AZaz:
                m1 += m1m2[i]
                m2 += "_"
                m3 += m2m3[i]
            else:
                m1+= " "
                m2+= " "
                m3+= " "

        elif m2m3[i] not in AZaz:
            if m1m3[i] in AZaz and m1m2[i] in AZaz:
                m1 += "_"
                m2 += m1m2[i]
                m3 += m1m3[i]
            else:
                m1+= " "
                m2+= " "
                m3+= " "
                
    return m1, m2, m3

In [3]:
c1hex = "315c4eeaa8b5f8aaf9174145bf43" #28char
c2hex = "234c02ecbbfbafa3ed18510abd11"
c3hex = "32510ba9a7b2bba9b8005d43a304"

m1m2hex = xor_strhex(c1hex, c2hex)
m1m3hex = xor_strhex(c1hex, c3hex)
m2m3hex = xor_strhex(c2hex, c3hex)

print(m1m2hex)
print(m1m3hex)
print(m2m3hex)


print("............................")


processedm1m2 = process2analyze(m1m2hex)
processedm1m3 = process2analyze(m1m3hex)
processedm2m3 = process2analyze(m2m3hex)

print(processedm1m2)
print(processedm1m3)
print(processedm2m3)

print("............................")


m1, m2, m3 = partial_decode(processedm1m2, processedm1m3, processedm2m3)

print(m1)
print(m2)
print(m3)

12104c06134e5709140f104f0252
030d45430f07430341171c061c47
111d09451c49140a55180c491e15
............................
  L  NW    O R
  EC  C A    G
   E I  U  I  
............................
  _C N_ A  O _
  LE _W U  _ R
  E_ IC _  I G
